# What's the fuzz all about?
## Dynamic Unit Testing in Python

* Moritz Gronbach, Blue Yonder
* EuroPython 2015, Bilbao, Spain
* [https://github.com/mogron/ep_dyntest](https://github.com/mogron/ep_dyntest)

# What is dynamic unit testing

In [2]:
from math import sqrt


def fib(n):
    """Computes the n-th Fibonacci number.
    fib(0) == fib(1) == 1
    fib(n) == fib(n - 1) + fib(n - 2)
    1, 1, 2, 3, 5, 8, ..."""
    sqrt_5 = sqrt(5)
    p = (1 + sqrt_5) / 2
    q = 1/p
    return int((p**n + q**n) / sqrt_5 + 0.5)

In [3]:
def test_fib():
    assert(fib(1) == 1)
    assert(fib(2) == 1)
    assert(fib(3) == 2)
    assert(fib(6) == 8)
    assert(fib(50) == 12586269025)
    print("Tests passed!")

test_fib()

Tests passed!


In [4]:
from hypothesis import given
from hypothesis.strategies import integers
from hypothesis import Settings, Verbosity


# settings to increase chances of a smooth presentation
Settings.default.derandomize = True
Settings.default.max_iterations = 50
Settings.default.timeout = 20
Settings.database = None

In [6]:
@given(integers(min_value=3, max_value=1000),
       settings=Settings(verbosity=Verbosity.verbose))
def test_fib_reccurence(n):
    assert(fib(n) == fib(n-1) + fib(n-2))
    
test_fib_reccurence()

Trying example: test_fib_reccurence(n=69)
Trying example: test_fib_reccurence(n=901)
Trying example: test_fib_reccurence(n=568)
Traceback (most recent call last):
  File "/home/DataScience/virtualenv/local/lib/python2.7/site-packages/hypothesis/core.py", line 494, in is_template_example
    always_print=settings.max_shrinks <= 0
  File "/home/DataScience/virtualenv/local/lib/python2.7/site-packages/hypothesis/executors/executors.py", line 26, in default_executor
    return function()
  File "/home/DataScience/virtualenv/local/lib/python2.7/site-packages/hypothesis/core.py", line 340, in run
    return test(*args, **kwargs)
  File "<ipython-input-6-63b1d755da88>", line 4, in test_fib_reccurence
    assert(fib(n) == fib(n-1) + fib(n-2))
AssertionError

Trying example: test_fib_reccurence(n=3)
Trying example: test_fib_reccurence(n=285)
Traceback (most recent call last):
  File "/home/DataScience/virtualenv/local/lib/python2.7/site-packages/hypothesis/core.py", line 494, in is_template_exa

AssertionError: 